# Demo - Query, Key, and Value Vectors

In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [2]:
tokens = ["she", "gave", "the", "dog", "its", "food"]

In [3]:
torch.manual_seed(2)

embedding_dim = 12
embeddings = {
    token: torch.randn(embedding_dim)
    for token in tokens
}

In [4]:
W_Q = torch.randn(embedding_dim, embedding_dim)
W_K = torch.randn(embedding_dim, embedding_dim)
W_V = torch.randn(embedding_dim, embedding_dim)

Q = {t: embeddings[t] @ W_Q for t in tokens}
K = {t: embeddings[t] @ W_K for t in tokens}
V = {t: embeddings[t] @ W_V for t in tokens}

In [5]:
query_token = "she"
query_vector = Q[query_token]

In [6]:
scores = {}

for token in tokens:
    scores[token] = torch.dot(query_vector, K[token]).item()


In [7]:
df = pd.DataFrame({
    "Token": tokens,
    "Query·Key Score": [scores[t] for t in tokens]
}).sort_values("Query·Key Score", ascending=False)

df

,Token,Query·Key Score
1,gave,70.584274
0,she,44.846226
2,the,8.416949
3,dog,-6.216842
4,its,-13.181318
5,food,-13.569026


In [8]:
raw_scores = torch.tensor([scores[t] for t in tokens])
attention_weights = F.softmax(raw_scores, dim=0)

attention_df = pd.DataFrame({
    "Token": tokens,
    "Attention Weight": attention_weights.detach().numpy()
}).sort_values("Attention Weight", ascending=False)

attention_df

,Token,Attention Weight
1,gave,1.000000e+00
0,she,6.639075e-12
2,the,1.002476e-27
3,dog,4.422814e-34
4,its,4.178924e-37
5,food,2.835876e-37


In [9]:
updated_she = torch.zeros(embedding_dim)

for i, token in enumerate(tokens):
    updated_she += attention_weights[i] * V[token]


In [10]:
def cosine_similarity(a, b):
    return F.cosine_similarity(a.unsqueeze(0), b.unsqueeze(0)).item()

print("Similarity (original → updated 'she'):",
      cosine_similarity(embeddings["she"], updated_she))


Similarity (original → updated 'she'): 0.27983707189559937
